In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/user-chatbot-data/final7.csv


In [8]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [9]:
import torch
from transformers import AutoTokenizer, AutoModel, GPT2Tokenizer, GPT2LMHeadModel
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

2024-08-15 14:24:36.612683: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-15 14:24:36.612787: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-15 14:24:36.752859: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:

checkpoint = "openai-community/gpt2"

# model = AutoModel.from_pretrained(checkpoint)

# model_2 = GPT2LMHeadModel.from_pretrained(checkpoint)

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# tokenizer_2 = GPT2Tokenizer.from_pretrained(checkpoint)

In [11]:
data = pd.read_csv('/kaggle/input/user-chatbot-data/final7.csv')
data

,user,chatbot
0,I always mess things up. Why can't I do anythi...,This sounds like an example of 'all-or-nothing...
1,I'm terrified of public speaking. How can I ov...,CBT would approach this with exposure therapy ...
2,I can't stop worrying about the future. How do...,CBT uses several techniques for this. One is '...
3,I always assume people are judging me negative...,"This is an example of 'mind reading', another ..."
4,I've been feeling really low lately. What can ...,CBT emphasizes the connection between thoughts...
...,...,...
2059,My grandson's step-mother sends him to school ...,<p>Absolutely not!&nbsp;</p><p>It is never in ...
2060,My boyfriend is in recovery from drug addictio...,<p>I'm sorry you have tension between you and ...
2061,The birth mother attempted suicide several tim...,"<p>The true answer is, ""no one can really say ..."
2062,I think adult life is making him depressed and...,<p>How do you help yourself to believe you req...


In [ ]:
# data_ = data.replace(to_replace=r'(<p>|</p>|\|\n)', value=' ', regex=True)

In [12]:
import re
import html

def clean_text(text):
    # Remove HTML tags using a regular expression
    text = re.sub(r'<[^>]+>', ' ', text)

    # Unescape HTML entities (e.g., &nbsp;, &amp;, etc.)
    text = html.unescape(text)

    # 3. Remove any remaining non-word characters or HTML entities
    # This will remove anything that isn't a letter, number, or whitespace
    text = re.sub(r'[^\w\s]', ' ', text)

    # 4. Remove multiple spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Example usage
dirty_text = """
<p>This is a sample text with some HTML tags<br> and some &nbsp; non-breaking spaces. 
Here is a link: <a href="https://example.com">example</a>. And here is some more &amp; text.</p>
"""

cleaned_text = clean_text(dirty_text)
print(cleaned_text)


This is a sample text with some HTML tags and some non breaking spaces Here is a link example And here is some more text


In [13]:
cleaned_data = data.map(clean_text)

In [ ]:
# doc = '[Q]'+' '+data_['user'] + '\n' + '[A]'+ ' '+data_['chatbot']
# len_data = doc.map(lambda x: len(x))
# len_data

In [14]:
doc_cleaned = '[Q]'+' '+cleaned_data['user'] + '\n' + '[A]'+ ' '+cleaned_data['chatbot']
len_data = doc_cleaned.map(lambda x: len(x))
len_data

0        321
1        411
2        377
3        344
4        361
        ... 
2059     944
2060    1030
2061    1081
2062     569
2063     264
Length: 2064, dtype: int64

In [ ]:
doc_cleaned[2054]

In [ ]:
len_data[len_data>1500]

In [ ]:
# doc_decoded = doc.map(lambda x: x.encode('utf-8').decode('utf-8'))

In [ ]:
doc_decoded[2054]

In [15]:
doc_str = "\n\n".join(doc_cleaned.astype(str))

with open('/kaggle/working/document.txt','w') as f:
    f.write(doc_str)

In [16]:
def load_dataset(file_path, tokenizer, block_size=128):
    dataset = TextDataset(tokenizer=tokenizer,
                         file_path=file_path,
                         block_size=block_size)
    return dataset

In [17]:
def load_data_collator(tokenizer, mlm=False):
    data_collator = DataCollatorForLanguageModeling(
                    tokenizer=tokenizer,
                    mlm = mlm)
    return data_collator

In [18]:
def train(train_file_path, model_name,
         output_dir,
         overwrite_output_dir,
         per_device_train_batch_size,
         num_train_epochs,
         save_steps):
    
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    train_dataset = load_dataset(train_file_path, tokenizer)
    data_collator = load_data_collator(tokenizer)
    
    model = GPT2LMHeadModel.from_pretrained(model_name)
    
    
    tokenizer.save_pretrained(output_dir)
    model.save_pretrained(output_dir)
    
        
    
#     training_args = TrainingArguments(
#     output_dir = output_dir,
#     overwrite_output_dir = overwrite_output_dir,
#     per_device_train_batch_size = per_device_train_batch_size,
#     num_train_epochs = num_train_epochs,
#     save_steps = 10000000,
#     save_total_limit = 1
#     )

    training_args = TrainingArguments(
    output_dir = output_dir,
    overwrite_output_dir = overwrite_output_dir,
    per_device_train_batch_size = per_device_train_batch_size,
    num_train_epochs = num_train_epochs,
    save_steps = 10000000,
    save_total_limit = 1,
    save_strategy = "steps",
    logging_steps = 10000000,  # Set this to a large number to reduce logging frequency
    evaluation_strategy = "no",  # Disable evaluation to prevent evaluation-triggered saves
    )
    
    trainer = Trainer(
    model = model,
    args = training_args,
    data_collator = data_collator,
    train_dataset = train_dataset)
    
    trainer.train()
    trainer.save_model()

In [19]:
train_file_path = '/kaggle/working/document.txt'
output_dir = '/kaggle/working'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 50
save_steps = 10000000

In [20]:
import os
import warnings
warnings.filterwarnings("ignore")

os.environ['WANDB_DISABLED'] = 'true'


train(train_file_path = train_file_path,
     model_name = checkpoint,
     output_dir = output_dir,
     overwrite_output_dir = overwrite_output_dir,
     per_device_train_batch_size = per_device_train_batch_size,
     num_train_epochs = num_train_epochs,
     save_steps = save_steps
     )

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss


In [23]:
model_check = GPT2LMHeadModel.from_pretrained(output_dir)

In [28]:
from IPython.display import FileLink
FileLink(r'/kaggle/working/checkpoint-12450.zip')

/kaggle/working/checkpoint-12450.zip

In [30]:
import zipfile
import os
from IPython.display import FileLink

def zip_dir(directory, file_name = 'directory.zip'):
    """
    zip all the files in a directory
    
    Parameters
    _____
    directory: str
        directory needs to be zipped, defualt is current working directory
        
    file_name: str
        the name of the zipped file (including .zip), default is 'directory.zip'
        
    Returns
    _____
    Creates a hyperlink, which can be used to download the zip file)
    """
    os.chdir(directory)
    zip_ref = zipfile.ZipFile(file_name, mode='w')
    for folder, _, files in os.walk(directory):
        for file in files:
            if file_name in file:
                pass
            else:
                zip_ref.write(os.path.join(folder, file))

    return FileLink(file_name)

In [47]:
# zip_dir(directory = output_dir, file_name = 'directory.zip')

In [37]:
from IPython.display import FileLink
FileLink(r'checkpoint-12450/model.safetensors')

/kaggle/working/checkpoint-12450/model.safetensors

In [38]:
def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer

def generate_text(model_path, sequence, max_length):
    
    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    print(tokenizer.decode(final_outputs[0], skip_special_tokens=True))

In [45]:
doc_cleaned[1]

'[Q] I m terrified of public speaking How can I overcome this fear\n[A] CBT would approach this with exposure therapy and cognitive restructuring Start by identifying your specific fears about public speaking Then gradually expose yourself to speaking situations starting small like speaking in front of a mirror and working up Meanwhile challenge and reframe the negative thoughts associated with public speaking'

In [51]:
sequence2 = "[Q] I am feeling very sad, blaming myself for not being confident"
max_len = 100
generate_text(output_dir, sequence2, max_len) 

[Q] I am feeling very sad, blaming myself for not being confident enough and not being good enough in my daily life I feel like every day I don t feel good enough or doing anything right I feel like the people in my family have done nothing to try to stop me and I feel as if I have no one whom I can trust
[A] Do you have some information that can help explain to you what is going on in your life Is there a hierarchy of expectations for your life
